In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing import image

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
data_path = '/home/vego/Documents/skripsi/program/dataset/CK+48'
data_dir_list = os.listdir(data_path)
print(data_dir_list)

In [ ]:
aug_image = ImageDataGenerator(rescale=1./255,
                                validation_split=0.3,
                                rotation_range=20,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.2,  
                                horizontal_flip=True,
                                shear_range=0.2,
                                fill_mode='nearest')

In [ ]:
size = 48
batch = 32

train_data = aug_image.flow_from_directory(data_path,
                                          #color_mode='grayscale',
                                          target_size=(size,size),
                                          batch_size=batch,
                                          class_mode='categorical',
                                          subset='training')

valid_data = aug_image.flow_from_directory(data_path,
                                          #color_mode='grayscale',
                                          target_size=(size,size),
                                          batch_size=batch,
                                          class_mode='categorical',
                                          subset='validation',
                                          shuffle=False)

test_data = aug_image.flow_from_directory(data_path,
                                         #color_mode='grayscale',
                                         target_size=(size,size),
                                         batch_size=1,
                                         shuffle=False)

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,MaxPooling2D,BatchNormalization

In [ ]:
model=Sequential()

model.add(Conv2D(filters=32,kernel_size=4,activation="relu",input_shape=(48,48,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32,kernel_size=4,activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=4,activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(7,activation="softmax"))
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# compile model dengan 'adam' optimize loss function 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
             optimizer=Adam(learning_rate=0.001),
             metrics=['accuracy'])

In [ ]:
# save_model = 'my_model.h5'

# checkpoint = ModelCheckpoint(save_model, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# callback_list=[checkpoint]

# latih model dengan model.fit
hist = model.fit(train_data,
          batch_size=batch,
          epochs=100, # tambahkan epoch jika akurasi model belum optimal
          validation_data=valid_data,
          #callbacks=callback_list,
          verbose=2)
hist

In [ ]:
from keras.models import load_model
import numpy as np

In [ ]:
model_path = '/home/vego/Documents/skripsi/program/my_model.h5'
model = load_model(model_path)

In [ ]:
# menggunakan model untuk mengklasifikasikan gambar kamar

import numpy as np 
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# masukkan file nama gambar 
path = '/home/vego/Documents/skripsi/program/dataset/uji/angry/S026_003_00000013.png'
img = tf.keras.preprocessing.image.load_img(path, target_size=(48,48))
imgplot = plt.imshow(img)
x = image.img_to_array(img)
x = tf.image.resize(x, [48,48])
x = np.expand_dims(x, axis=0)

classes={
    0: 'Angry',
    1: 'Contempt',
    2: 'Disgust',
    3: 'Fear',
    4: 'Happy',
    5: 'Sadness',
    6: 'Surprise'
}
images = np.vstack([x])
pred = model.predict(images)

class_names = classes.values()
class_name = list(class_names)

result =class_name[np.argmax(pred)]
print(path)
print(result)

In [ ]:
import numpy as np 
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']

epochs = range(len(train_acc))

plt.plot(epochs,train_loss,'r', label='train_loss')
plt.plot(epochs,val_loss,'b', label='val_loss')
plt.title('train_loss vs val_loss')
plt.legend()
plt.figure()

plt.plot(epochs,train_acc,'r', label='train_acc')
plt.plot(epochs,val_acc,'b', label='val_acc')
plt.title('train_acc vs val_acc')
plt.legend()
plt.figure()